In [1]:
# %% Import
import pandas as pd
import sys
import os
import pymc3 as pm
import theano.tensor as tt
import arviz as az

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from forseti.datproc import translate_categorical, extract_sensitive


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# %% Translate Categorical Attributes
df = pd.read_csv('data/adult.csv')
tmp, codes = translate_categorical(df)

In [3]:
# %% Extract sensitive attributes
sensattr = ['gender', 'race']
sensitive, features = extract_sensitive(tmp, sensattr)

In [4]:
y = features['income']
X = features.drop(['income', 'fnlwgt'], axis=1)

In [19]:
y[:100]

0     0
1     0
2     1
3     1
4     0
     ..
95    0
96    0
97    0
98    1
99    0
Name: income, Length: 100, dtype: int8

In [21]:
with pm.Model() as model:
    W_shared = pm.Data("W_obs", X['workclass'][:100])
    D_shared = pm.Data("D_obs", y[:100])
    S_shared = pm.Data("S_obs", sensitive['gender'][:100])

    # Fair Labels 
    Fair_p = pm.Beta("Fair_p", alpha=1, beta=1, shape=2)
    Df = pm.Categorical("Df", p=Fair_p)

    # Sensitive Attributes
    Sensitive_p = pm.Beta("Sensitive_p", alpha=1, beta=1, shape=2)
    S = pm.Categorical("S", p=Sensitive_p, observed=S_shared)

    # Data Labels P(D | S, Df)
    Data_p = pm.Beta("Data_p", alpha=1, beta=1, shape=2)
    D_beta = pm.Normal('D_beta', mu=0, sd=1)
    mu = tt.dot(tt.dot(Df, S), D_beta)
    p = tt.nnet.softmax(mu)
    D = pm.Categorical("D", p=p, observed=D_shared)

    # P(W | Df, S)
    beta = pm.Normal('beta', mu=0, sd=1)
    mu = tt.dot(tt.dot(Df, S), beta)
    p = tt.nnet.softmax(mu)
    W = pm.Categorical('W', p=p, observed=W_shared)
    trace = pm.sample()
    idata = az.from_pymc3(trace)

# %%

ERROR (theano.graph.opt): Optimization failure due to: LocalOptGroup(local_useless_fill,local_useless_alloc,local_subtensor_make_vector,local_useless_elemwise,local_useless_inc_subtensor,local_useless_slice,local_subtensor_of_alloc,local_useless_inc_subtensor_alloc,local_useless_rebroadcast,local_join_1,local_join_empty,local_join_make_vector,local_useless_switch,local_useless_tile,local_useless_split,local_useless_reshape,local_useless_elemwise_comparison,local_useless_reduce,local_view_op,local_merge_alloc,local_useless_topk)
ERROR (theano.graph.opt): node: ViewOp(Elemwise{Cast{int64}}.0)
ERROR (theano.graph.opt): TRACEBACK:
ERROR (theano.graph.opt): Traceback (most recent call last):
  File "C:\tools\Anaconda3\envs\forseti\lib\site-packages\theano\graph\opt.py", line 2060, in process_node
    fgraph.replace_all_validate_remove(repl_pairs, reason=lopt, remove=remove)
  File "C:\tools\Anaconda3\envs\forseti\lib\site-packages\theano\graph\toolbox.py", line 612, in replace_all_validate_

TypeError: expected type_num 1 (NPY_INT8) got 9
Apply node that caused the error: Elemwise{mul,no_inplace}(InplaceDimShuffle{x,x}.0, InplaceDimShuffle{x,0}.0, InplaceDimShuffle{x,x}.0)
Toposort index: 8
Inputs types: [TensorType(int64, (True, True)), TensorType(int8, row), TensorType(float64, (True, True))]
Inputs shapes: [(1, 1), (1, 100), (1, 1)]
Inputs strides: [(8, 8), (800, 8), (8, 8)]
Inputs values: [array([[0]], dtype=int64), 'not shown', array([[0.]])]
Outputs clients: [[Softmax(Elemwise{mul,no_inplace}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [ ]:
with model:
    # change the value and shape of the data
    pm.set_data(
        {
            "S_obs": sensitive['gender'][1001],
            "W_obs": X['workclass'][1001],

            # use dummy values with the same shape:
            "Fair_p": [0],
        }
    )

    post_pred = pm.sample_posterior_predictive(idata.posterior)

post_pred["Df"]

KeyError: 'S_obs'

In [ ]:
X['workclass'][1001]

4